# Storage (flash)

Broadly, [Docker](https://docs.docker.com/storage/) containers distinguish different "types" of data storage.

## Owned by the container

That's the default and includes all operating system related files, installed software (compilers, etc). 

Although it's possible to make changes (e.g. with `apt-get` run inside the container), those changes do not persist between updates and hence should be avoided. Best practice is not to modify files "owned" by the container. 

This part of storage is "private" to the container and invisible to other containers in the same app.

## Mounted into the container (volumes)

Storage can also be "mounted" from the "host", the underlying operating system that runs the Docker app, *ide49*. In *ide49*, `/home/iot` and `/service-config` are mounted. 

The same volumes can be mounted into several containers (at the same or different locations). This allows sharing data. For example, all volumes are mounted in the `duplicati` (backup) container. 

Likewise, configuration files, e.g. for `nginx` (webserver), are mounted in `/service-config` in the `code-server` and `jupyter` containers for editing, and at the proper path in the various containers that need configuration (e.g. `/etc/nginx`).

Changes to volumes persist between container updates. 

## Samba fileshare

In addition to the types of storage discussed above, *ide49* can also be configured to mount `/home/iot` from a Samba file server.

Uses include sharing the same data between several *ide49* instances or in situations where a shared file server, perhaps with centralized backup, is preferred.

For convenience, the entire home directory is mounted. This can cause some undesired behavior. For example, databases frequently use status files (e.g. locks) used by the running instance. Sharing those with another device (running perhaps a different instance of the same database) results in conflicts.

To prevent this, the local iot volume is always available at `/service-config/iot-home`. Unlike `/home/iot`, this copy is never "hidden" by a Samba mount. *ide49* is configured to store status information (e.g. for `jupyter`) in `/service-config/iot-home` rather than `/home/iot`.

## Python packages (pip)

In general, customizing *ide49* is accomplished by modifying the application (Dockerfiles etc). Fortunately that's straightforward and can be accomplished right [from within *ide49*](customize.ipynb). Installing additional linux packages (`apt-get`) should use this route.

Python packages are a special case. App development invariably runs into situations where a needed package is not preinstalled in *ide49*. Because of this, the `jupyter` container in *ide49* supports installing Python packages by the user. Effectively using `pip` from the command line "just works". 

This is how:

1. The Python interpreter itself and core packages are owned by root and installed at a location owned by the container. Modify *ide49* to make changes to these (e.g. upgrade the interpreter).

2. The Dockerfile installs packages (e.g. `jupyterlab`) as root. They are available on the Python path.

3. Packages installed by the user are saved to `/service-config/iot-home/.local` (configured by the environment variable `PYTHONUSERBASE`). This location persists between app updates. It is never shared/mounted by Samba, avoiding the possibility of package incompatibilities (e.g. different processor architectures, `aarch64` and `amd64`).

Python is configured to look for packages in all these locations.

In [1]:
!python -m site

sys.path = [
    '/home/iot/iot49.org/docs/ide/reference',
    '/home/iot/projects/robot/code-pi',
    '/usr/local/lib/python38.zip',
    '/usr/local/lib/python3.8',
    '/usr/local/lib/python3.8/lib-dynload',
    '/service-config/iot-home/.local/lib/python3.8/site-packages',
    '/usr/local/lib/python3.8/site-packages',
]
USER_BASE: '/service-config/iot-home/.local' (exists)
USER_SITE: '/service-config/iot-home/.local/lib/python3.8/site-packages' (exists)
ENABLE_USER_SITE: True


In [2]:
!echo $PATH

/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


## Failed attempt to install packages in container as non-root

* Problem(s):
    * Cannot get the iot-kernel recognized in Jupyter

In general, customizing *ide49* is accomplished by modifying the application (Dockerfiles etc). Fortunately that's straightforward and can be accomplished right [from within *ide49*](customize.ipynb). Installing additional linux packages (`apt-get`) should use this route.

Python packages are a special case. App development invariably runs into situations where a needed package is not preinstalled in *ide49*. Because of this, the `jupyter` container in *ide49* supports installing Python packages by the user. Effectively using `pip` from the command line "just works". 

This is how:

1. The Python interpreter itself and core packages are owned by root and installed at a location owned by the container. Modify *ide49* to make changes to these (e.g. upgrade the interpreter).

2. Packages preinstalled by the container (e.g. `jupyterlab`) are located in `/opt/python_base`. This folder is part of the container (i.e. overwritten each time the app is updated), but "owned" by user `iot`. This is to prevent problems with Python packages installed as the `root` user.

3. Packages installed by the user are saved to `/service-config/iot-home/.local` (configured by the environment variable `PYTHONUSERBASE`). This location persists between app updates. It is never shared/mounted by Samba, avoiding the possibility of package incompatibilities (e.g. different processor architectures, `aarch64` and `amd64`).

Python is configured to look for packages in all these locations.